In [1]:
!pip install kaggle
!python3 -m pip install pandas

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.3/63.3 kB 1.3 MB/s eta 0:00:002.5 MB/s eta 0:00:01
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 155.3/155.3 kB 2.3 MB/s eta 0:00:00 MB/s eta 0:00:01
  Using cached python_dateutil-2.8.2-py2.py3-none-any.whl (247 kB)
  Using cached requests-2.28.2-py3-none-any.whl (62 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.1/77.1 kB 1.4 MB/s eta 0:00:00 MB/s eta 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 140.9/140.9 kB 1.5 MB/s eta 0:00:003.1 MB/s eta 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.2/78.2 kB 2.1 MB/s eta 0:00:003.0 MB/s eta 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.5/61.5 kB 873.8 kB/s eta 0:00:0031m1.2 MB/s eta 0:00:01
  Using cached charset_normalizer-3.1.0-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (199 kB)
  Created wheel for kaggle: filename=kaggle-1.5.13-py3-none-any.whl size=77716 sha256=6d9db4019157eb638666ce91cba77d5ad528598249

In [2]:
#{"username":"anguis1","key":"8e5aa869bd2b736636371bd2e59902b3"}

import os
import zipfile
import kaggle

os.environ['KAGGLE_USERNAME'] = 'anguis1'
os.environ['KAGGLE_KEY'] = '8e5aa869bd2b736636371bd2e59902b3'

#Nom du dataset
dataset = "vishalsubbiah/pokemon-images-and-types"

#Téléchargement du Dataset
kaggle.api.dataset_download_files(dataset)

with zipfile.ZipFile('pokemon-images-and-types.zip', 'r') as zip_ref:
    zip_ref.extractall('pokemon')

ModuleNotFoundError: No module named 'kaggle'

In [10]:
import urllib.request
import json
import pandas as pd

#On charge le fichier csv
df = pd.read_csv('./pokemon/pokemon.csv')

#On crée un dictionnaire associant le nom du pokemon à son image
pokemon_images = {}
for filename in os.listdir("./pokemon/images/images"):
    if filename.endswith(".png"):
        name = filename.split(".")[0]
        pokemon_images[name] = os.path.join("pokemon/images/images", filename)

# Ajouter une colonne "image" au dataframe
df["image"] = df["Name"].apply(lambda x: pokemon_images.get(x, None))

# Supprimer les lignes sans image
df = df.dropna(subset=["image"])

# Afficher le dataframe
print(df.head())

         Name  Type1   Type2                                 image
0   bulbasaur  Grass  Poison   pokemon/images/images/bulbasaur.png
1     ivysaur  Grass  Poison     pokemon/images/images/ivysaur.png
2    venusaur  Grass  Poison    pokemon/images/images/venusaur.png
3  charmander   Fire     NaN  pokemon/images/images/charmander.png
4  charmeleon   Fire     NaN  pokemon/images/images/charmeleon.png


In [29]:
import numpy as np
from sklearn.cluster import KMeans

def get_dominant_colors(image_path, n_colors):
    # Charger l'image et la convertir en une matrice numpy
    img = Image.open(image_path)
    img = np.array(img)

    # Obtenir un tableau de pixels en deux dimensions
    pixels = img.reshape(-1, 3)
    #-1 car l'argument s'adapte à la taille de l'image et 3 pour le RGB
    
    # Créer un objet de clustering
    kmeans = KMeans(n_clusters=n_colors)

    # Entraîner le modèle
    kmeans.fit(pixels)

    # Récupérer les couleurs dominantes
    dominant_colors = kmeans.cluster_centers_

    # Convertir les couleurs en entiers et les ajouter à une liste
    dominant_colors = [tuple(map(int, color)) for color in dominant_colors]
    
    return '#{:02x}{:02x}{:02x}'.format(dominant_colors)

In [30]:
# import random
import random

def satisfied_or_not():
    
    #Fonction pour générer un avis aléatoire (positif ou négatif) de l'utilisateur pour pouvoir effectuer des tests
    
    return random.choice(['Yes', 'No'])

In [3]:
import os
import csv
import json
from PIL import Image

# Chemin du dossier contenant les images
folder_path = './pokemon/images/images'

# Liste des extensions de fichier que nous voulons inclure dans l'analyse
extensions = ['.jpg', '.jpeg', '.png']

# Initialisation de la liste pour stocker les informations de chaque image
list_images_info = []

# Ouverture du fichier CSV
with open('./pokemon/pokemon.csv') as csv_file:
    csv_reader = csv.reader(csv_file)
    next(csv_reader) # Ignorer la première ligne (en-tête)
    for row in csv_reader:
        # Récupère les informations du CSV
        pokemon_name = row[0]
        type1 = row[1]
        try :
            type2 = row[2]
        except Exception as e:
            type2 = 'None'
            continue

        filename = row[0]+'.png'
    
        try:
            # Vérifie si le fichier est une image avec une extension spécifiée
            if any(filename.lower().endswith(ext) for ext in extensions):
                # Chemin complet de l'image
                filepath = os.path.join(folder_path, filename)
                # Ouvre l'image avec PIL
                with Image.open(filepath) as img:
                    # Stocke les informations dans un dictionnaire
                    images_info = {'filename': filename,
                                   'pokemon_name': pokemon_name,
                                   'type1': type1,
                                   'type2': type2,
                                   'size': os.path.getsize(filepath),
                                   'k-means_regions': get_dominant_colors(filepath,1),
                                   'favorite_utilisateur1': satisfied_or_not(),
                                   'favorite_utilisateur2': satisfied_or_not(),
                                   'favorite_utilisateur3': satisfied_or_not(),
                                   'favorite_utilisateur4': satisfied_or_not()}
                    # Ajoute le dictionnaire à la liste
                    list_images_info.append(images_info)
        except Exception as e:
            filename = row[0]+'.jpg' #Si le fichier n'existe pas en .png, on teste de le trouver en .jpg
    
            try:
                # Vérifie si le fichier est une image avec une extension spécifiée
                if any(filename.lower().endswith(ext) for ext in extensions):
                    # Chemin complet de l'image
                    filepath = os.path.join(folder_path, filename)
                    # Ouvre l'image avec PIL
                    with Image.open(filepath) as img:
                        # Stocke les informations dans un dictionnaire
                        images_info = {'filename': filename,
                                       'pokemon_name': pokemon_name,
                                       'type1': type1,
                                       'type2': type2,
                                       'size': os.path.getsize(filepath),
                                       'k-means_regions': get_dominant_colors(filepath,1),
                                       'favorite_utilisateur1': satisfied_or_not(),
                                       'favorite_utilisateur2': satisfied_or_not(),
                                       'favorite_utilisateur3': satisfied_or_not(),
                                       'favorite_utilisateur4': satisfied_or_not()}
                        # Ajoute le dictionnaire à la liste
                        list_images_info.append(images_info)
            except Exception as e:
                print(f"Erreur lors de l'extraction des informations de {row[0]}: {e}")
                continue

# Écriture du résultat dans un fichier JSON
with open('images_info.json', 'w') as json_file:
    json.dump(list_images_info, json_file)

Erreur lors de l'extraction des informations de bulbasaur: [Errno 2] No such file or directory: './pokemon/images/images/bulbasaur.jpg'
Erreur lors de l'extraction des informations de ivysaur: [Errno 2] No such file or directory: './pokemon/images/images/ivysaur.jpg'
Erreur lors de l'extraction des informations de venusaur: [Errno 2] No such file or directory: './pokemon/images/images/venusaur.jpg'
Erreur lors de l'extraction des informations de charizard: [Errno 2] No such file or directory: './pokemon/images/images/charizard.jpg'
Erreur lors de l'extraction des informations de butterfree: [Errno 2] No such file or directory: './pokemon/images/images/butterfree.jpg'
Erreur lors de l'extraction des informations de weedle: [Errno 2] No such file or directory: './pokemon/images/images/weedle.jpg'
Erreur lors de l'extraction des informations de kakuna: [Errno 2] No such file or directory: './pokemon/images/images/kakuna.jpg'
Erreur lors de l'extraction des informations de beedrill: [Errno

In [ ]:
rom sklearn import tree
import pandas as pd
from sklearn.preprocessing import LabelEncoder
import graphviz
import pydotplus
from IPython.display import Image, display


with open('./data_images.json') as d:
    file_contents = d.read()
data = json.loads(file_contents)
datadict={}
resultdict={}

favorite_all=[
        "favorite_utilisateur2",
        "favorite_utilisateur3",
        "favorite_utilisateur4",
             "k-means_regions"]

favorite=["favorite_utilisateur1"]
for keyi in data[0].keys(): 
    if keyi not in favorite_all:
        if keyi in favorite:
            resultdict[keyi]=[]
        else:
            datadict[keyi]=[]

for dicti in data:
    for keyi in dicti.keys():
        if keyi not in favorite_all:
            if keyi in favorite:
                resultdict[keyi].append(dicti[keyi])
            else:
                datadict[keyi].append(dicti[keyi])


#creating dataframes
dataframe = pd.DataFrame(datadict)
resultframe = pd.DataFrame(resultdict)

print(dataframe)
print(resultframe)

#generating numerical labels
le1 = LabelEncoder() #On associe un chiffre avec le texte du tag
dataframe['orientation'] = le1.fit_transform(dataframe['orientation'])

le2 = LabelEncoder()
dataframe['format'] = le2.fit_transform(dataframe['format'])

le3 = LabelEncoder()
dataframe['size'] = le3.fit_transform(dataframe['size'])

le4 = LabelEncoder()
dataframe['date_created'] = le4.fit_transform(dataframe['date_created'])

le5 = LabelEncoder() #On associe un chiffre avec le texte du tag
dataframe['filename'] = le5.fit_transform(dataframe['filename'])

le6 = LabelEncoder()
dataframe['camera_model'] = le6.fit_transform(dataframe['camera_model'])
"""
le7 = LabelEncoder()
dataframe['k-means_regions'] = le7.fit_transform(dataframe['k-means_regions'])
"""

le7 = LabelEncoder()
resultframe['favorite_utilisateur1'] = le7.fit_transform(resultframe['favorite_utilisateur1'])
"""
le9 = LabelEncoder()
resultframe['favorite_utilisateur2'] = le9.fit_transform(resultframe['favorite_utilisateur3'])

le10 = LabelEncoder()
resultframe['favorite_utilisateur3'] = le10.fit_transform(resultframe['favorite_utilisateur3'])

le11 = LabelEncoder()
resultframe['favorite_utilisateur4'] = le11.fit_transform(resultframe['favorite_utilisateur4'])
"""
#Use of decision tree classifiers
dtc = tree.DecisionTreeClassifier()
dtc = dtc.fit(dataframe, resultframe)

dot_data = tree.export_graphviz(dtc, out_file=None,
      feature_names=dataframe.columns,
      filled=True, rounded=True, 
      class_names =
      le5.inverse_transform(
        resultframe.favorite.unique())
     ) 
graph = graphviz.Source(dot_data) 

pydot_graph = pydotplus.graph_from_dot_data(dot_data)
img = Image(pydot_graph.create_png())
display(img)